In [1]:
import open3d as o3d
import numpy as np
import matplotlib.cm as plt

pcd_dir = "../extracted_pcl/"

In [2]:
def readFile(FileName):
    path = pcd_dir + FileName
    print("Reading in file")
    pcd = o3d.io.read_point_cloud(path)
    print(pcd)
    print(np.asarray(pcd.points))
    o3d.visualization.draw_geometries([pcd])
    # alpha = 1
    # print(f"alpha={alpha:.3f}")
    # mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
    # mesh.compute_vertex_normals()
    # o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
    # with o3d.utility.VerbosityContextManager(
    #     o3d.utility.VerbosityLevel.Debug) as cm:
    #     labels = np.array(
    #         pcd.cluster_dbscan(eps=0.02, min_points=10, print_progress=True))

    # max_label = labels.max()
    # print(f"point cloud has {max_label + 1} clusters")
    # colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    # colors[labels < 0] = 0
    # pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    # o3d.visualization.draw_geometries([pcd],
    #                                 zoom=0.455,
    #                                 front=[-0.4999, -0.1659, -0.8499],
    #                                 lookat=[2.1813, 2.0619, 2.0999],
    #                                 up=[0.1204, -0.9852, 0.1215])
    # pcl, ind = pcd.remove_radius_outlier(nb_points=30, radius=1)
    # hull, _ = pcl.compute_convex_hull()
    # hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
    # hull_ls.paint_uniform_color((1, 0, 0))
    # o3d.visualization.draw_geometries([pcl, hull_ls])


In [3]:

readFile("94856000.pcd")

Reading in file
PointCloud with 3763 points.
[[ 10.1      -31.700001   0.1     ]
 [ 10.5      -31.9        0.1     ]
 [ 10.9      -31.9        0.1     ]
 ...
 [ 27.299999 -15.3        0.1     ]
 [ 27.700001 -15.3        0.1     ]
 [ 28.299999 -15.3        0.1     ]]
alpha=1.000


RuntimeError: [1;31m[Open3D ERROR] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh[0;m

In [8]:
mesh = o3d.geometry.Image.create_pyramid(5,False)
o3d.visualization.draw_geometries(mesh)

TypeError: create_pyramid(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.geometry.Image, num_of_levels: int, with_gaussian_filter: bool) -> List[open3d.cpu.pybind.geometry.Image]

Invoked with: 5, True